In [38]:
import pandas as pd

## Data Processing (Phase 2)

In [59]:
app_path = "/Users/noahrae-grant/nohrg/MRFRS-stats-analysis/data/raw/adopt_applications/"
applications = []
for i in range(5):
    applications.append(pd.read_excel(app_path + f"applications202{i+1}.xlsx"))

In [60]:
application_df = pd.concat(applications)

In [61]:
for app in applications:
    print(app.shape)

(1892, 12)
(1700, 12)
(1844, 12)
(1747, 12)
(1068, 12)


In [62]:
application_df.shape

(8251, 12)

In [63]:
application_df.columns

Index(['Application ID', 'Application Type', 'Application Received Date/Time',
       'Status', 'Application Status Date/Time', 'Application Note',
       'Animal ID', 'Person ID', 'City', 'State', 'County', 'Zip'],
      dtype='object')

In [64]:
application_df["Animal ID"].value_counts()

Animal ID
—    8251
Name: count, dtype: int64

In [65]:
application_df = application_df.drop(columns=["Animal ID"])

In [151]:
#application_df["City"] = application_df["City"].str.lower().str.strip()

In [66]:
application_df[application_df["City"] == 383]

,Application ID,Application Type,Application Received Date/Time,Status,Application Status Date/Time,Application Note,Person ID,City,State,County,Zip
1099,4449396,Cat Adoption Application,"09/01/2023, 4:39 pm",Accepted,"09/02/2023, 12:16 pm",—,MER-P-19604,383,MA,NaN,01843


In [67]:
sorted([str(x) for x in application_df["City"].unique().tolist()])

['01830',
 '01832',
 '03858',
 '23',
 '383',
 'ALLSTON',
 'AMESBURY',
 'AMHERST',
 'ANDOVER',
 'ARLINGTON',
 'Abington',
 'Active',
 'Acton',
 'Acushnet',
 'Allenstown',
 'Allston',
 'Alstead, NH',
 'Alton',
 'Alton Bay',
 'Amebury',
 'Ames',
 'Amesburt',
 'Amesbury',
 'Amesbury ',
 'Amesury',
 'Amey',
 'Amherst',
 'Amsbury',
 'Andover',
 'Andovwr',
 'Apt 3D',
 'Arlington',
 'Ashaway',
 'Ashburnham',
 'Ashland',
 'Astoria',
 'Athol',
 'Atkinskon',
 'Atkinson',
 'Atkinson ',
 'Atkinson no 03811 ',
 'Attleboro',
 'Auburn',
 'Auburn ',
 'Auburndale',
 'Ayer',
 'BELMONT',
 'BEVERLY',
 'BILLERICA',
 'BOSTON',
 'BOXFORD',
 'BRIGHTON',
 'BROOKLINE',
 'Babson Park',
 'Baldwinville',
 'Baltimore',
 'Barre',
 'Barrington',
 'Barrington ',
 'Beacon',
 'Bedfird',
 'Bedford',
 'Bedford, Mass',
 'Belchertown',
 'Belfast',
 'Bellingham',
 'Belmont',
 'Bennington',
 'Berkley',
 'Berlin',
 'Berwick',
 'Beverly',
 'Beverly ',
 'Biddeford',
 'Billerica',
 'Billerica ',
 'Bloomfield',
 'Bolton',
 'Boscawe

In [68]:
from pyzipcode import ZipCodeDatabase
zcdb = ZipCodeDatabase()

In [69]:
def zco_to_city(elem):
    if len(str(elem)) > 5:
        elem = str(elem)[:5]
        if elem == 'O1701': elem = '01701'
        if elem == "O3865": elem = '03865'
        try:
            elem = int(elem)
        except ValueError:
            elem = int(elem[:4])
    try:
        city = zcdb[elem].city
        return city
    except KeyError:
        return elem

application_df["City_Fixed"] = application_df['Zip'].apply(zco_to_city)

In [53]:
application_df[application_df['City_Fixed'].str.isnumeric() == True][['City', 'State', 'City_Fixed']].value_counts()

Series([], Name: count, dtype: int64)

In [52]:
def fix_nulls(elem):
    # manually assuming that what people put in their city is correct
    # and that they typed their zip wrong
    # or that the module I used is incomplete
    elem = str(elem)
    if elem == "O3865": return "Plaistow"
    if elem.isnumeric() is False:
        return elem
    if elem in ['03861', '3861']: return 'Lee'
    if elem == '03823': return 'Madbury'
    if elem in ['01804', '08301']: return 'Burlington'
    if elem == '01802': return 'Woburn'
    if elem == '01878': return 'Tyngsboro'
    if elem == '03285': return 'Thornton'
    if elem == '03682': return 'Nashua' # best guess
    if elem == '03919': return 'Danville'
    if elem == '03877': return 'Somersworth'
    if elem in ['02952', '1972', '01972']: return 'Salisbury'
    if elem == '01979': return 'Salem'
    if elem in ['09150', '11912', '1959', '01959']: return 'Newburyport'
    if elem == '01443': return 'Leominster'
    if elem == '03994': return 'Kittery Point'
    if elem in ['00000', '91832']: return 'Haverhill'
    if elem == '09130': return 'Gloucester'
    if elem == '03093': return 'Eliot'
    if elem in ['02920', '02923']: return 'Danvers'
    if elem in ['1920', '1932']: return 'Byfield'
    if elem in ['1986', '01986']: return 'West Newbury'
    if elem == '4909': return 'Wells'
    if elem == '92467': return 'Newton'
    if elem == '1992': return 'Lynn'
    if elem == '4036': return 'Kennebunkport'
    if elem == '8134': return 'Groveland'
    if elem == '2034': return 'Durham'
    if elem == '1836': return 'Chelmsford'
    if elem == '77719': return 'Joshua Tree'
    return elem

application_df['City_Fixed'] = application_df['City_Fixed'].apply(fix_nulls)

In [54]:
def update_zip(elem):
    # manually assuming that what people put in their city is correct
    # and that they typed their zip wrong
    # or that the module I used is incomplete
    elem = str(elem)
    if elem.isnumeric() is False:
        return elem
    if elem in ['03861', '3861']: return '03861'
    if elem == '03823': return '03823'
    if elem in ['01804', '08301']: return '01803'
    if elem == '01802': return '01801'
    if elem == '01878': return '01879'
    if elem == '03285': return 'Thornton'
    if elem == '03682': return 'Nashua' # best guess
    if elem == '03919': return 'Danville'
    if elem == '03877': return 'Somersworth'
    if elem in ['02952', '1972', '01972']: return 'Salisbury'
    if elem == '01979': return 'Salem'
    if elem in ['09150', '11912', '1959', '01959']: return 'Newburyport'
    if elem == '01443': return 'Leominster'
    if elem == '03994': return 'Kittery Point'
    if elem in ['00000', '91832']: return 'Haverhill'
    if elem == '09130': return 'Gloucester'
    if elem == '03093': return 'Eliot'
    if elem in ['02920', '02923']: return 'Danvers'
    if elem in ['1920', '1932']: return 'Byfield'
    if elem in ['1986', '01986']: return 'West Newbury'
    if elem == '4909': return 'Wells'
    if elem == '92467': return 'Newton'
    if elem == '1992': return 'Lynn'
    if elem == '4036': return 'Kennebunkport'
    if elem == '8134': return 'Groveland'
    if elem == '2034': return 'Durham'
    if elem == '1836': return 'Chelmsford'
    if elem == '77719': return 'Joshua Tree'
    return elem

application_df['City_Fixed'] = application_df['City_Fixed'].apply(fix_nulls)

In [55]:
application_df['City_Fixed'].value_counts()

City_Fixed
Haverhill         590
Amesbury          477
Newburyport       380
Salisbury         255
Boston            175
                 ... 
Fresno              1
Ashaway             1
New London          1
Mont Vernon         1
Pomfret Center      1
Name: count, Length: 510, dtype: int64

In [56]:
application_df[(application_df['City_Fixed'] == "Salisbury") & (application_df['City'] != "Salisbury")]["City"].unique()

array(['salisbury', 'SALISBURY', 'Seabrook', 'saliabury', 'sailsbury',
       'Burlington', 'Salisbury ', 'Salisbury, MA 01952, USA'],
      dtype=object)

In [57]:
application_df["City"] = application_df["City_Fixed"]
application_df = application_df.drop(columns=["City_Fixed"])

In [58]:
application_df.to_csv("/Users/noahrae-grant/nohrg/MRFRS-stats-analysis/data/clean/applications.csv")

Calendar data was cleaned by hand, since it was a very small dataset comparatively.

Of course, I needed to redo the adopter data to fix the city names there.

In [170]:
adopters = pd.read_csv("/Users/noahrae-grant/nohrg/MRFRS-stats-analysis/data/clean/adopter.csv", index_col=0)

In [171]:
adopters.columns

Index(['Date Of Adoption', 'Animal ID', 'Animal Name', 'Species',
       'Primary Breed', 'Sex', 'Altered', 'Est. Birthdate', 'Microchip Number',
       'By (User)', 'Outcome Subtype', 'Adopter ID', 'City', 'State', 'Zip'],
      dtype='object')

In [173]:
adopters["City_Fixed"] = adopters["Zip"].apply(zco_to_city)

In [184]:
adopters[adopters['City_Fixed'].str.isnumeric() == True][['City', 'State', 'City_Fixed']].value_counts()

Series([], Name: count, dtype: int64)

In [183]:
adopters["City_Fixed"] = adopters["City_Fixed"].apply(fix_nulls)

In [187]:
adopters["City"] = adopters["City_Fixed"]
adopters = adopters.drop(columns=["City_Fixed"])

In [188]:
adopters.to_csv("/Users/noahrae-grant/nohrg/MRFRS-stats-analysis/data/clean/adopter.csv")